In [1]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve

In [2]:
grouped_purchased = pd.read_csv('cvs_data_file/online-retail.csv', header = 0)
grouped_purchased.head()

,CustomerID,StockCode,Quantity
0,12346,23166,1
1,12347,16008,24
2,12347,17021,36
3,12347,20665,6
4,12347,20719,40


In [3]:
customers = list(np.sort(grouped_purchased.CustomerID.unique())) 
products = list(grouped_purchased.StockCode.unique()) 
quantity = list(grouped_purchased.Quantity) 

rows = grouped_purchased.CustomerID.astype('category', categories = customers).cat.codes 
cols = grouped_purchased.StockCode.astype('category', categories = products).cat.codes 
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

C:\Users\Office\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  """
C:\Users\Office\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  import sys


In [4]:
print(purchases_sparse.A)

[[ 1  0  0 ...  0  0  0]
 [ 0 24 36 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


In [5]:
import random

In [6]:
def make_train(ratings, pct_test = 0.2):
    test_set = ratings.copy() 
    test_set[test_set != 0] = 1 
    training_set = ratings.copy() 
    nonzero_inds = training_set.nonzero() 
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) 
    random.seed(0) 
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) 
    samples = random.sample(nonzero_pairs, num_samples) 
    user_inds = [index[0] for index in samples] 
    item_inds = [index[1] for index in samples] 
    training_set[user_inds, item_inds] = 0 
    training_set.eliminate_zeros() 
    return training_set, test_set, list(set(user_inds))  

In [7]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

In [8]:
item_lookup = pd.read_csv('cvs_data_file/item_lookup.csv', header = 0)
item_lookup.head()

,StockCode,Description
0,85123A,WHITE HANGING HEART T-LIGHT HOLDER
1,71053,WHITE METAL LANTERN
2,84406B,CREAM CUPID HEARTS COAT HANGER
3,84029G,KNITTED UNION FLAG HOT WATER BOTTLE
4,84029E,RED WOOLLY HOTTIE WHITE HEART.


In [9]:
def get_items_purchased(customer_id, mf_train, customers_list, products_list, item_lookup):
    cust_ind = np.where(customers_list == customer_id)[0][0] 
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] 
    prod_codes = products_list[purchased_ind] 
    return item_lookup.loc[item_lookup.StockCode.isin(prod_codes)]

In [10]:
import pickle

with open('saved_model','rb') as f:
            saved_model=pickle.load(f)
            model=saved_model['model']

C:\Users\Office\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [11]:
from sklearn.metrics.pairwise import cosine_similarity

def display_item_to_items_recommendations(model,item_id):

    products_arr = np.array(products) 
    
    item_id = np.where(products_arr == item_id)[0][0]
    print(item_id)
  
    return item_lookup['Description'][cosine_similarity(
            model.item_embeddings)[item_id].argsort()][-5:][::-1]


display_item_to_items_recommendations(model, str('85123A'))

896


896          RING OF ROSES BIRTHDAY CARD
1492                       POTTERING MUG
1822       FRENCH GARDEN SIGN BLUE METAL
1833    4 LILY  BOTANICAL DINNER CANDLES
1213              PAPER CHAIN KIT EMPIRE
Name: Description, dtype: object

In [14]:
def display_recommended_items(model, data, user_ids):
    customers_arr = np.array(customers)
    products_arr = np.array(products) 
    print("customers_arr",customers_arr)
    user_id = np.where(customers_arr == user_ids)[0][0]
    
    n_users, n_items = data.shape
#     known_positives = get_items_purchased(user_ids, product_train, customers_arr, products_arr, item_lookup)
    known_positives = item_lookup['Description'][data.tocsr()[user_id].indices]
    known_positives_df = pd.DataFrame(data=known_positives)
    print(known_positives_df,'\n','######################################')            
    
    
    scores = model.predict(user_id, np.arange(n_items))

    
    top_items = item_lookup['Description'][np.argsort(-scores)]
    print(top_items)
    
    
display_recommended_items(model, product_train, 18282)

customers_arr [12346 12347 12348 ... 18282 18283 18287]
                              Description
34    ROUND SNACK BOXES SET OF4 WOODLAND 
44                MINI PAINT SET VINTAGE 
68      LUNCH BOX WITH CUTLERY RETROSPOT 
69        PACK OF 72 RETROSPOT CAKE CASES
629          HEADS AND TAILS SPORTING FUN
814           ANT WHITE WIRE HEART SPIRAL
815                FEATHER PEN,COAL BLACK
1398          HOME SWEET HOME METAL SIGN 
1518   ASS DES PHONE SPONGE CRAFT STICKER
1731        SMALL GLASS SUNDAE DISH CLEAR
2753         TRAVEL CARD WALLET KEEP CALM 
 ######################################
34      ROUND SNACK BOXES SET OF4 WOODLAND 
42               MINI JIGSAW CIRCUS PARADE 
1592                        ANIMAL STICKERS
44                  MINI PAINT SET VINTAGE 
43                     MINI JIGSAW SPACEBOY
814             ANT WHITE WIRE HEART SPIRAL
1449                 COLUMBIAN CANDLE ROUND
231          TV DINNER TRAY VINTAGE PAISLEY
214          STRAWBERRY CERAMIC TRINKET BOX
887